In [3]:
import xarray as xr
from sklearn_xarray import wrap
from dask_ml.preprocessing import StandardScaler
import intake
from dask.distributed import Client
# client = Client(processes=False)
# client = Client(threads_per_worker=4, n_workers=10, processes=False)
client = Client("tcp://127.0.0.1:60612")
client

Client Scheduler: tcp://127.0.0.1:60612 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 32 Cores: 32 Memory: 67.48 GB


In [2]:
# client.close()

In [11]:
catalog_url = 'https://raw.githubusercontent.com/obidam/m2poc2019/master/m2poc_data_catalog.yml'
ds = intake.Catalog(catalog_url).argo_global_homogeneous_sdl.read_chunked()
print('This dataset holds: %.3f GB' % (ds.nbytes / 1e9))
print(ds)

This dataset holds: 0.626 GB
<xarray.Dataset>
Dimensions:    (DEPTH: 302, N_PROF: 128910)
Coordinates:
  * DEPTH      (DEPTH) float32 0.0 -5.0 -10.0 -15.0 ... -1495.0 -1500.0 -1505.0
    LATITUDE   (N_PROF) float32 dask.array<shape=(128910,), chunksize=(67010,)>
    LONGITUDE  (N_PROF) float32 dask.array<shape=(128910,), chunksize=(67010,)>
    TIME       (N_PROF) datetime64[ns] dask.array<shape=(128910,), chunksize=(64455,)>
Dimensions without coordinates: N_PROF
Data variables:
    BRV2       (N_PROF, DEPTH) float32 dask.array<shape=(128910, 302), chunksize=(67010, 302)>
    DBINDEX    (N_PROF) float64 dask.array<shape=(128910,), chunksize=(67010,)>
    PSAL       (N_PROF, DEPTH) float32 dask.array<shape=(128910, 302), chunksize=(67010, 302)>
    SIG0       (N_PROF, DEPTH) float32 dask.array<shape=(128910, 302), chunksize=(67010, 302)>
    TEMP       (N_PROF, DEPTH) float32 dask.array<shape=(128910, 302), chunksize=(67010, 302)>
Attributes:
    Title:             VENTHSAL-IA
    conv

In [4]:
feat_0 = ds.TEMP.sel(DEPTH=0, method='nearest')
feat_1 = ds.TEMP.sel(DEPTH=-1000, method='nearest')
RES = xr.concat([feat_0, feat_1], dim='feature').chunk(chunks={'N_PROF':10000})
RES = RES.rename('ARGO')
# Remove obsolete attributes:
for a in RES.attrs.copy(): 
    del RES.attrs[a]
# Add feature names:
RES.attrs['feature_0'] = "%s at %0.2fm depth"%(feat_0.name,feat_0['DEPTH'])
RES.attrs['feature_1'] = "%s at %0.2fm depth"%(feat_1.name,feat_1['DEPTH'])
# We're good:
print(RES)

<xarray.DataArray 'ARGO' (feature: 2, N_PROF: 128910)>
dask.array<shape=(2, 128910), dtype=float32, chunksize=(1, 10000)>
Coordinates:
    LATITUDE   (N_PROF) float32 dask.array<shape=(128910,), chunksize=(10000,)>
    LONGITUDE  (N_PROF) float32 dask.array<shape=(128910,), chunksize=(10000,)>
    TIME       (N_PROF) datetime64[ns] dask.array<shape=(128910,), chunksize=(10000,)>
    DEPTH      (feature) float32 dask.array<shape=(2,), chunksize=(2,)>
Dimensions without coordinates: feature, N_PROF
Attributes:
    feature_0:  TEMP at 0.00m depth
    feature_1:  TEMP at -1000.00m depth


In [5]:
X = xr.DataArray(RES.values.T, dims=['sample','feature'])
print(X.values.shape)

(128910, 2)


In [7]:
scaler = wrap(StandardScaler(), sample_dim='sample')
%time scaler.fit(X)
print(scaler.estimator_.mean_)
print(scaler.estimator_.scale_)

CPU times: user 2 ms, sys: 1 ms, total: 3 ms
Wall time: 1.53 ms
[-9.5641155e-08  2.3862941e-07]
[1.        0.9999999]


In [8]:
scaler.transform(X)

<xarray.DataArray (sample: 128910, feature: 2)>
array([[-1.551811, -1.29166 ],
       [-1.82948 , -1.280055],
       [-1.588464, -1.29136 ],
       ...,
       [-1.656578, -0.882205],
       [-0.824594, -0.895902],
       [-1.067999, -0.828581]], dtype=float32)
Dimensions without coordinates: sample, feature

In [9]:
print(X)
print('This dataset holds: %.3f GB' % (X.nbytes / 1e9))

<xarray.DataArray (sample: 128910, feature: 2)>
array([[-1.551811, -1.29166 ],
       [-1.82948 , -1.280055],
       [-1.588464, -1.29136 ],
       ...,
       [-1.656578, -0.882205],
       [-0.824594, -0.895902],
       [-1.067999, -0.828581]], dtype=float32)
Dimensions without coordinates: sample, feature
This dataset holds: 0.001 GB


In [12]:
scaler = wrap(StandardScaler(), sample_dim='N_PROF')
%time scaler.fit(ds)
print(scaler.estimator_.mean_)
print(scaler.estimator_.scale_)

CPU times: user 151 ms, sys: 11 ms, total: 162 ms
Wall time: 20.2 s


AttributeError: 'EstimatorWrapper' object has no attribute 'estimator_'